In [1]:
from utils import DataSet
import utils.augmentation as aug
import utils.helpers as helpers

In [2]:
import torch, torchvision 
import albumentations as A
import cv2
import numpy as np 
import os, random, time, json, math

from torch.utils.data import DataLoader
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from sklearn.model_selection import train_test_split
from mean_average_precision import MetricBuilder
from tqdm.auto import tqdm
from detectron2.utils.logger import setup_logger

from ImageEnhancement import MSRCR, FUSION, CLAHE

In [16]:
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog, build_detection_train_loader
from detectron2.data import detection_utils as utils
import detectron2.data.transforms as T

In [21]:
BATCH_SIZE = 4 # increase / decrease according to GPU memeory
RESIZE_TO = 800 # resize the image for training and transforms
NUM_EPOCHS = 15 # number of epochs to train for
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# training images and XML files directory
SEED = 42
TEST_RATIO = 0.1 # for train/test split

MAPPING = {
    'Kartoffel': 0,
    'Fish': 1,
    'Cnidaria':2   
}

# whether to visualize images after crearing the data loaders
VISUALIZE_TRANSFORMED_IMAGES = False
# location to save model and plots
OUT_DIR = '/home/anton/Documents/Thesis/temResults/'
SAVE_PLOTS_EPOCH = 5 # save loss plots after these many epochs
SAVE_MODEL_EPOCH = 5 # save model after these many epochs

PREPARE_TEST_DATA = True

IMAGE_DIRECTORY = "images"

In [5]:
base_dir = "/home/anton/Downloads/image_annotator/linux_v1.4.3/data/oldclean/"

imgs, labels = helpers.image_and_label_paths(base_dir, image_dir = "images", label_dir = "labels")
inputs_train, inputs_valid, targets_train, targets_valid = train_test_split(imgs, labels, test_size=TEST_RATIO, random_state=SEED)


In [6]:
# Train DataSet:
train_dataset = DataSet(inputs_train, 
                            targets_train, 
                            use_cache          = False,
                            mapping            = MAPPING,
                            random_enhancement = False,
                            use_detectron      = True
                            )



# Validation DataSet:
validation_dataset = DataSet(inputs_valid, 
                                targets_valid, 
                                use_cache          = False,
                                mapping            = MAPPING, 
                                random_enhancement = False,
                                use_detectron      = True
                            )          


In [7]:
for d, data in zip(["train", "val"], [train_dataset, validation_dataset]):
    DatasetCatalog.register("fish_" + d, lambda f=data, d=d: f.get_data_dicts(name = d))
    MetadataCatalog.get("fish_" + d).set(thing_classes=list(MAPPING.keys()))

fish_metadata = MetadataCatalog.get("fish_train")


In [11]:
counter = 0
dataset_dicts = train_dataset.get_data_dicts(name  = "train")
for d in random.sample(dataset_dicts, 10):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=fish_metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    cv2.imwrite("./hi_{}.png".format(counter), out.get_image()[:, :, ::-1])
    counter += 1

  0%|          | 0/102 [00:00<?, ?it/s]

loading train dataset


In [17]:
class CustomMapper():
    def __init__(self, transformlist, random_enhancement = False, enhancement_directories = ["images", "msrcr", "clahe", "fusion"]):
        self._transformlist = transformlist
        self.random_enhancemant = random_enhancement
        self.random_enhacement_directories = enhancement_directories

    def __call__(self, dataset_dict):
        dataset_dict = copy.deepcopy(dataset_dict)  # it will be modified by code below
    
        #HERE COMES THE STUFF FOR READING IMAGES WITH RANDOM ENHANCEMENTS(from different directories)
        if self.random_enhancemant:
            dataset_dict["file_name"] = dataset_dict["file_name"].replace("images", random.choice(self.random_enhacement_directories))
        image = utils.read_image(dataset_dict["file_name"], format="BGR")
        image, transforms = T.apply_transform_gens(self._transformlist, image)
        dataset_dict["image"] = torch.as_tensor(image.transpose(2, 0, 1).astype("float32"))

        annos = [
            utils.transform_instance_annotations(obj, transforms, image.shape[:2])
            for obj in dataset_dict.pop("annotations")
        ]
        instances = utils.annotations_to_instances(annos, image.shape[:2])
        dataset_dict["instances"] = utils.filter_empty_instances(instances)
        return dataset_dict

class MyTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR)
        return mAPEvaluator(dataset_name, outpath = output_folder)

    @classmethod
    def build_train_loader(cls, cfg):
        transformlist = [T.Resize((csg.RESIZE, csg.RESIZE)),
                      T.RandomFlip(prob=0.5, horizontal=False, vertical=True),
                      T.RandomFlip(prob=0.5, horizontal=True, vertical=False), 
                      T.RandomBrightness(0.8,1.2),
                      T.RandomCrop("relative", (0.6,0.6))
                      ]
        return build_detection_train_loader(cfg, mapper=CustomMapper(transformlist, random_enhancement = cfg.RANDOM_ENHANCEMENT))

In [22]:
OUTDIR = "/content/drive/MyDrive/FISHimg/detectron_msrcr/"
cfg = get_cfg()

cfg.OUTPUT_DIR = OUT_DIR
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("fish_train",)
cfg.DATASETS.TEST = ("fish_val",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = BATCH_SIZE
cfg.SOLVER.BASE_LR = 0.003  #LR
cfg.RESIZE = RESIZE_TO

cfg.SOLVER.MAX_ITER = NUM_EPOCHS * len(inputs_train) / BATCH_SIZE
cfg.TEST.EVAL_PERIOD = 250

cfg.SOLVER.STEPS = []        
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(MAPPING.keys())
cfg.RANDOM_ENHANCEMENT = False

In [24]:
#iterations = np.arange(1, cfg.SOLVER.MAX_ITER, 1)
iterations = [cfg.SOLVER.MAX_ITER]
for it in iterations:
    print(it)
    cfg.SOLVER.MAX_ITER = int(it)
    os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
    trainer = MyTrainer(cfg) 
    trainer.resume_or_load(resume=True)
    trainer.train()

    xx = OUTDIR+"model_final.pth"
    xx2 = OUTDIR+"model_final_{}it.pth".format(it)
    !cp {xx} {xx2}

1.0


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx